# Single shortest path

In [55]:
import numpy as np
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler

## Define graph

(1st vertice, 2nd vertice, weight)

In [56]:
edge = np.array([(0, 1, 5), (0, 2, 1), (2, 1, 2), (1, 3, 3), (3, 2, 4)])
vertices = 4
start = 0
end = 3

In [57]:
#edge = np.array([(0, 1, 8), (1, 3, 5), (0, 2, 2), (2, 1, 4), (2, 4, 7), (4, 3, 3)])
#vertices = 5
#start = 0
#end = 3

In [58]:
#edge = np.array([(0, 1, 5), (0, 2, 5), (1, 0, 5), (1, 2, 2), (1, 3, 2),
#                 (2, 0, 5), (2, 1, 2), (2, 3, 10), (3, 1, 2), (3, 2, 10)])
#vertices = 4
#start = 0
#end = 3

## Create array from edge array

In [59]:
m = 0
edges = len(edge)
edge_names = {}
edge_weights = {}
graph = np.zeros((vertices,vertices))
for i,e in enumerate(edge):
    graph[e[0],e[1]] = e[2]
    m += e[2]
    edge_names[i] = str(e[0]) + '-' + str(e[1])
    edge_weights[edge_names[i]] = e[2]

    
print(graph)
print('Max count:',m)

[[0. 5. 1. 0.]
 [0. 0. 0. 3.]
 [0. 2. 0. 0.]
 [0. 0. 4. 0.]]
Max count: 15


## Create QUBO from edge array

Based on "Directed Edge-Based Approach" from Kraus & McCollum (2020) Solving the Network Shortest Path Problem on a Quantum Annealer.

https://doi.org/10.1109/TQE.2020.3021921


Constraints: 

- the source vertex has one more edge leaving it than entering it
- the terminal vertex has one more edge entering it than leaving it
- every other vertex has as many edges entering it as it has leaving it.



In [60]:
Q = np.zeros((edges,edges))

# Include weights of edges

for i,e in enumerate(edge):
    Q[i][i] = e[2]

# set constraints for starting node, ending node and other nodes for incoming and outgoing edges

for x in range(vertices):
    ec = np.zeros(edges)                # Has edge i this vertex as an input (+1) or and output (-1). 
    for i,e in enumerate(edge):
        if e[0]==x:
            ec[i] = 1
        if e[1]==x:
            ec[i] = -1

    for i in range(edges):              # Loop through all edge combinations, and add penalty if both are incoming
        for j in range(i+1, edges):     #   or outgoing to vertex x, and negative penalty if one is incoming and 
            Q[i][j] += ec[i]*ec[j]*2*m  #   one is outgoing.

    for i in range(edges):
        Q[i][i] += ec[i]*ec[i]*m        # Add penalty to diagonal if vertex is input or output of edge. 
        if x==end:                      # If we are in ending vertex, add penalty if vertex is input for edge, 
            Q[i][i] += ec[i]*2*m        #   negative penalty if vertex is output for edge.
        if x==start:                    # Similar mirrored operation if we are in starting vertex. 
            Q[i][i] -= ec[i]*2*m

print(edge_names)
print(Q)
print(sum(sum(Q)))

{0: '0-1', 1: '0-2', 2: '2-1', 3: '1-3', 4: '3-2'}
[[  5.  30.  30. -30.   0.]
 [  0.   1. -30.   0.  30.]
 [  0.   0.  32. -30. -30.]
 [  0.   0.   0.   3. -30.]
 [  0.   0.   0.   0.  64.]]
45.0


## Create BQM

In [61]:
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
bqm = bqm.relabel_variables(edge_names, inplace=False)

## Solving with simulator

In [62]:
simSampler = SimulatedAnnealingSampler()
sampleset = simSampler.sample(bqm, num_reads=20)
print(sampleset)

   0-1 0-2 1-3 2-1 3-2 energy num_oc.
1    0   1   1   1   0  -24.0       1
2    0   1   1   1   0  -24.0       1
3    0   1   1   1   0  -24.0       1
4    0   1   1   1   0  -24.0       1
5    0   1   1   1   0  -24.0       1
7    0   1   1   1   0  -24.0       1
8    0   1   1   1   0  -24.0       1
11   0   1   1   1   0  -24.0       1
12   0   1   1   1   0  -24.0       1
13   0   1   1   1   0  -24.0       1
14   0   1   1   1   0  -24.0       1
15   0   1   1   1   0  -24.0       1
0    1   0   1   0   0  -22.0       1
6    1   0   1   0   0  -22.0       1
9    1   0   1   0   0  -22.0       1
10   1   0   1   0   0  -22.0       1
16   1   0   1   0   0  -22.0       1
17   1   0   1   0   0  -22.0       1
18   1   0   1   0   0  -22.0       1
19   1   0   1   0   0  -22.0       1
['BINARY', 20 rows, 20 samples, 5 variables]


In [63]:
r = sampleset.first.sample
r = dict((k, edge_weights[k]) for k, v in r.items() if v==1)
print('Edges:',r)
print('Total weights:', sum(r.values()))

Edges: {'0-2': 1, '1-3': 3, '2-1': 2}
Total weights: 6


Here shortest path should be 0-2, 2-1, 1-3. And total weights 6. 

## Solving with real thing

In [16]:
dwSampler = EmbeddingComposite(DWaveSampler())
sampleset2 = dwSampler.sample(bqm, num_reads=50)
print(sampleset2) 

  0-1 0-2 1-3 2-1 3-2 energy num_oc. chain_.
0   0   1   1   1   0  -24.0      26     0.0
1   1   0   1   0   0  -22.0      23     0.0
2   0   1   0   0   0    1.0       1     0.0
['BINARY', 3 rows, 50 samples, 5 variables]


In [17]:
time = sampleset2.info['timing']['qpu_access_time'] / 1000
print('QPU time used (ms): ', time)

QPU time used (ms):  20.96797


In [18]:
r = sampleset2.first.sample
r = dict((k, edge_weights[k]) for k, v in r.items() if v==1)
print('Edges:',r)
print('Total weights:', sum(r.values()))

Edges: {'0-2': 1, '1-3': 3, '2-1': 2}
Total weights: 6


## Solving with hybrid sampler

In [19]:
dwSampler = LeapHybridSampler()
sampleset3 = dwSampler.sample(bqm)
print(sampleset3) 

  0-1 0-2 1-3 2-1 3-2 energy num_oc.
0   0   1   1   1   0  -24.0       1
['BINARY', 1 rows, 1 samples, 5 variables]


In [21]:
time = sampleset3.info['qpu_access_time'] / 1000
print('QPU time used (ms): ', time)

QPU time used (ms):  96.359


In [22]:
r = sampleset3.first.sample
r = dict((k, edge_weights[k]) for k, v in r.items() if v==1)
print('Edges:',r)
print('Total weights:', sum(r.values()))

Edges: {'0-2': 1, '1-3': 3, '2-1': 2}
Total weights: 6
